In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
%matplotlib inline
warnings.filterwarnings("ignore")

# Set seeds to make the experiment more reproducible.
from numpy.random import seed
seed(1)

In [3]:
data = pd.read_csv('BrentOilPrices.csv', index_col='Date')
data.head()

,Price
Date,
20-May-87,18.63
21-May-87,18.45
22-May-87,18.55
25-May-87,18.60
26-May-87,18.63


In [4]:
# from sklearn.preprocessing import MinMaxScaler
# values = data['Price'].values.reshape(-1,1)
# values = values.astype('float32')
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)
# train_size = int(len(scaled) * 0.7)
# test_size = len(scaled) - train_size
# train, test = scaled[0:train_size,:], scaled[train_size:len(scaled),:]
# print(len(train), len(test))

In [5]:
train_size = int(len(data) * 0.7)
test_size = len(data) - train_size
scaled = data['Price'].values
scaled = scaled.reshape(-1, 1)
train, test = scaled[0:train_size,:], scaled[train_size:len(scaled),:]
print(len(train), len(test))

5987 2567


In [6]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

In [7]:
look_back = 1
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

5986
2566


In [8]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [9]:
X_train.shape

(5986, 1, 1)

In [10]:
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=225, batch_size=100,
                    validation_data=(X_test, y_test), verbose=1, shuffle=False)

Epoch 1/225
60/60 [==============================] - 3s 13ms/step - loss: 32.0088 - val_loss: 70.5765
Epoch 2/225
60/60 [==============================] - 0s 5ms/step - loss: 26.0651 - val_loss: 62.4585
Epoch 3/225
60/60 [==============================] - 0s 5ms/step - loss: 19.3849 - val_loss: 55.8675
Epoch 4/225
60/60 [==============================] - 0s 6ms/step - loss: 15.2674 - val_loss: 52.3149
Epoch 5/225
60/60 [==============================] - 0s 6ms/step - loss: 14.9780 - val_loss: 51.0349
Epoch 6/225
60/60 [==============================] - 0s 6ms/step - loss: 13.8664 - val_loss: 49.1052
Epoch 7/225
60/60 [==============================] - 0s 6ms/step - loss: 13.1393 - val_loss: 47.2298
Epoch 8/225
60/60 [==============================] - 0s 8ms/step - loss: 13.0141 - val_loss: 45.8338
Epoch 9/225
60/60 [==============================] - 0s 6ms/step - loss: 12.4685 - val_loss: 44.3861
Epoch 10/225
60/60 [==============================] - 0s 5ms/step - loss: 12.1400 - val_lo

60/60 [==============================] - 0s 7ms/step - loss: 2.4035 - val_loss: 10.3920
Epoch 83/225
60/60 [==============================] - 0s 6ms/step - loss: 2.3809 - val_loss: 10.2769
Epoch 84/225
60/60 [==============================] - 0s 8ms/step - loss: 2.3672 - val_loss: 10.1607
Epoch 85/225
60/60 [==============================] - 0s 5ms/step - loss: 2.3497 - val_loss: 10.0610
Epoch 86/225
60/60 [==============================] - 0s 6ms/step - loss: 2.3203 - val_loss: 9.9151
Epoch 87/225
60/60 [==============================] - 0s 6ms/step - loss: 2.2964 - val_loss: 9.8402
Epoch 88/225
60/60 [==============================] - 0s 7ms/step - loss: 2.2755 - val_loss: 9.7564
Epoch 89/225
60/60 [==============================] - 0s 6ms/step - loss: 2.2473 - val_loss: 9.6481
Epoch 90/225
60/60 [==============================] - 0s 6ms/step - loss: 2.2372 - val_loss: 9.5788
Epoch 91/225
60/60 [==============================] - 0s 6ms/step - loss: 2.2360 - val_loss: 9.5222
Epoch 92/

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
yhat = model.predict(X_test)
plt.plot(yhat, label='predict')
plt.plot(y_test, label='true')
plt.legend()
plt.show()